In [ ]:
import pika;
import json;
import time;

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import re

# selenium=4.24.0

In [ ]:
MQ_server = '192.168.0.202'
driver_path = 'C:\\chromedriver-win64\\chromedriver.exe'
extension_path = 'C:\\chromedriver-win64\\bypass-paywalls-firefox-clean-master.crx'

In [ ]:
service = Service(executable_path=driver_path)
option = Options()
option.add_extension(extension=extension_path)
web = webdriver.Chrome(service=service,options=option)

In [ ]:
def webGet(url, title):
    web.get(url=url)
    textdata = web.find_elements(By.CLASS_NAME, 'evys1bk0')
    artifact = ''
    for data in textdata:
        artifact = artifact + data.text
    safe_filename = re.sub(r'[<>:"/\\|?*]', '_', title) + ".txt"
    with open(safe_filename,"w") as file:
        file.write(artifact)


In [ ]:
connection = pika.BlockingConnection(pika.ConnectionParameters(MQ_server))
channel = connection.channel()
channel.basic_qos(prefetch_count=1)
try:
    while True:
        method_frame, header_frame, body = channel.basic_get(queue='nytime', auto_ack=False)

        if method_frame:
            data = json.loads(body)
            print(f" [x] Received title={data['title']}.\n author={data['author']},\n time = {data['date']}")
            webGet(data['url'], data['title'])
            time.sleep(10)
            channel.basic_ack(delivery_tag=method_frame.delivery_tag)
        else:
            print(" [*] No messages in queue.")
            time.sleep(10)
except KeyboardInterrupt:
    print("\n [*] Exiting.")
finally:
    connection.close()